In [1]:
import pandas as pd
import copy

# Clases

In [46]:
class CursoMateria:
    def __init__(self, cursoMateria:str, bloques: list[str]):
        self.cursoMateria = cursoMateria
        self.materia = cursoMateria.split('-')[0]
        self.bloques = bloques
        self.profesPosibles = [] #profes que, segun disponibilidad y capacidad, podrían ser asignados para dar la materia
    def __repr__(self):
        return f'CursoMateria({self.cursoMateria}, {self.bloques}, {self.profesPosibles})'
    
    def __deepcopy__(self, memo):
        # 1. Create a new, uninitialized instance
        new_instance = type(self).__new__(type(self))

        # 2. Add to memo (CRITICAL for handling circular refs)
        memo[id(self)] = new_instance

        # 3. Deep-copy ALL attributes using copy.deepcopy and the memo
        #    This is the key change.
        new_instance.cursoMateria = copy.deepcopy(self.cursoMateria, memo)
        new_instance.materia = copy.deepcopy(self.materia, memo)
        new_instance.bloques = copy.deepcopy(self.bloques, memo)
        new_instance.profesPosibles = copy.deepcopy(self.profesPosibles, memo)
        
        return new_instance

    def prioridad(self):
        return len(self.profesPosibles)

In [69]:

class Profe:
    def __init__(self, id: str, bloquesParaAsignar: int, disponibilidad: dict, materiasPosibles: dict):
        self.id = id
        self.bloquesParaAsignar = bloquesParaAsignar
        self.disponibilidad = disponibilidad
        self.materiasPosibles = materiasPosibles
        self.cursosMateriasAsignados = []

    def __repr__(self):
        # A clearer representation to see the ID in the list output
        return f'Profe(ID={self.id})'

    def __str__(self):
        return f'Profe(ID={self.id}, BloquesParaAsignar={self.bloquesParaAsignar}, Disponibilidad={self.disponibilidad}, MateriasPosibles={self.materiasPosibles}, CursosMateriasAsignados={self.cursosMateriasAsignados})'

    def __deepcopy__(self, memo):
        # 1. Create a new, uninitialized instance
        new_instance = type(self).__new__(type(self))

        # 2. Add to memo (CRITICAL for handling circular refs)
        memo[id(self)] = new_instance

        # 3. Deep-copy ALL attributes using copy.deepcopy and the memo
        #    This is the key change.
        new_instance.id = copy.deepcopy(self.id, memo)
        new_instance.bloquesParaAsignar = copy.deepcopy(self.bloquesParaAsignar, memo)
        new_instance.disponibilidad = copy.deepcopy(self.disponibilidad, memo)
        new_instance.materiasPosibles = copy.deepcopy(self.materiasPosibles, memo)
        new_instance.cursosMateriasAsignados = copy.deepcopy(self.cursosMateriasAsignados, memo)
        
        return new_instance

    def podríaDarMateria(self, materia: CursoMateria) -> bool:
        # Verifica si el profe puede dar la materia considerando su capacidad y disponibilidad
        if self.materiasPosibles[materia.materia]:
            for bloque in materia.bloques:
                if self.disponibilidad[bloque] == False:
                    return False
            return True
        return False

# Carga y limpieza de datos

In [4]:
horarios = pd.read_csv('data/HorariosMaterias.csv')
#horarios = horarios.melt(id_vars=['Materia-Curso'], value_name='Bloque').drop(columns=['variable']).dropna()
horarios['Bloques'] = horarios.apply(lambda row: [row[col] for col in ['Bloque 1', 'Bloque 2'] if pd.notna(row[col])]  , axis=1)
horarios.drop(columns=['Bloque 1', 'Bloque 2'], inplace=True)
horarios

,Materia-Curso,Bloques
0,T7-A,"[M2, M3]"
1,T7-B,"[J3, J4]"
2,T7-C,"[M3, X4]"
3,T7-D,"[V3, V4]"
4,T7-E,"[V1, V2]"
5,T72-A,[J1]
6,T72-B,[L3]
7,T72-C,[V3]
8,T72-D,[J2]
9,T72-E,[J4]


In [5]:
disponibilidades = pd.read_csv('data/Disponibilidad.csv')
disponibilidades.drop(columns=['Unnamed: 0'], inplace=True)
# disponibilidades = disponibilidades.melt(id_vars=['PROF #', 'Bloques asignados'], var_name = 'Bloque', value_name='Disponibilidad')
# disponibilidades['Disponibilidad'] = disponibilidades['Disponibilidad'].apply(lambda x: True if x == 'x' else False)
disponibilidades['Disponibilidad'] = disponibilidades.apply(lambda row: {col: (True if row[col] == 'x' else False) for col in disponibilidades.columns[2:]}, axis=1)
disponibilidades.drop(columns=[col for col in disponibilidades.columns if col not in ['PROF #', 'Bloques asignados', 'Disponibilidad']], inplace=True)
disponibilidades

,PROF #,Bloques asignados,Disponibilidad
0,PROF 1,12,"{'L1': True, 'L2': False, 'L3': False, 'L4': F..."
1,PROF 2,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru..."
2,PROF 3,9,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru..."
3,PROF 4,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru..."
4,PROF 5,2,"{'L1': True, 'L2': True, 'L3': True, 'L4': Fal..."
5,PROF 6,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ..."
6,PROF 7,6,"{'L1': True, 'L2': True, 'L3': True, 'L4': Fal..."
7,PROF 8,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru..."
8,PROF 9,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ..."
9,PROF 10,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ..."


In [6]:
capacidades = pd.read_csv('data/Capacidades.csv')
capacidades.drop(columns=['Unnamed: 0'], inplace=True)
capacidades['Capacidad'] = capacidades.apply(lambda row: {col: (True if row[col] == 'x' else False) for col in capacidades.columns[1:]}, axis=1)
capacidades.drop(columns=[col for col in capacidades.columns if col not in ['PROF #', 'Capacidad']], inplace=True)
capacidades


,PROF #,Capacidad
0,PROF 1,"{'T7': False, 'T72': False, 'T1': False, 'T2':..."
1,PROF 2,"{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
2,PROF 3,"{'T7': False, 'T72': True, 'T1': True, 'T2': T..."
3,PROF 4,"{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
4,PROF 5,"{'T7': False, 'T72': True, 'T1': False, 'T2': ..."
5,PROF 6,"{'T7': False, 'T72': False, 'T1': False, 'T2':..."
6,PROF 7,"{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
7,PROF 8,"{'T7': True, 'T72': False, 'T1': False, 'T2': ..."
8,PROF 9,"{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
9,PROF 10,"{'T7': False, 'T72': False, 'T1': True, 'T2': ..."


In [7]:
profes = pd.merge(disponibilidades, capacidades, on='PROF #')
profes

,PROF #,Bloques asignados,Disponibilidad,Capacidad
0,PROF 1,12,"{'L1': True, 'L2': False, 'L3': False, 'L4': F...","{'T7': False, 'T72': False, 'T1': False, 'T2':..."
1,PROF 2,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru...","{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
2,PROF 3,9,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru...","{'T7': False, 'T72': True, 'T1': True, 'T2': T..."
3,PROF 4,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru...","{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
4,PROF 5,2,"{'L1': True, 'L2': True, 'L3': True, 'L4': Fal...","{'T7': False, 'T72': True, 'T1': False, 'T2': ..."
5,PROF 6,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ...","{'T7': False, 'T72': False, 'T1': False, 'T2':..."
6,PROF 7,6,"{'L1': True, 'L2': True, 'L3': True, 'L4': Fal...","{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
7,PROF 8,10,"{'L1': True, 'L2': True, 'L3': True, 'L4': Tru...","{'T7': True, 'T72': False, 'T1': False, 'T2': ..."
8,PROF 9,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ...","{'T7': False, 'T72': False, 'T1': True, 'T2': ..."
9,PROF 10,2,"{'L1': False, 'L2': False, 'L3': False, 'L4': ...","{'T7': False, 'T72': False, 'T1': True, 'T2': ..."


# Listas finales

Ahora que tengo los datos en formato que me sirva, lo armo en las clases, para aplicar los metodos...

In [57]:
materias = horarios.apply(lambda row: CursoMateria(row['Materia-Curso'], row['Bloques']), axis=1).tolist()
materias

[CursoMateria(T7-A, ['M2', 'M3'], []),
 CursoMateria(T7-B, ['J3', 'J4'], []),
 CursoMateria(T7-C, ['M3', 'X4'], []),
 CursoMateria(T7-D, ['V3', 'V4'], []),
 CursoMateria(T7-E, ['V1', 'V2'], []),
 CursoMateria(T72-A, ['J1'], []),
 CursoMateria(T72-B, ['L3'], []),
 CursoMateria(T72-C, ['V3'], []),
 CursoMateria(T72-D, ['J2'], []),
 CursoMateria(T72-E, ['J4'], []),
 CursoMateria(T1-A, ['L5', 'V3'], []),
 CursoMateria(T1-B, ['M4', 'J1'], []),
 CursoMateria(T1-C, ['X4', 'X5'], []),
 CursoMateria(T1-D, ['X2', 'J5'], []),
 CursoMateria(T1-E, ['M5', 'J2'], []),
 CursoMateria(T1-F, ['M3', 'V2'], []),
 CursoMateria(T1-G, ['L4', 'X2'], []),
 CursoMateria(T1-H, ['M3', 'J4'], []),
 CursoMateria(T1-I, ['X1', 'V4'], []),
 CursoMateria(T1-J, ['X3', 'J1'], []),
 CursoMateria(T1-K, ['L2', 'X1'], []),
 CursoMateria(T1-L, ['M4', 'J5'], []),
 CursoMateria(T1-M, ['L5', 'M1'], []),
 CursoMateria(T1-N, ['L1', 'V1'], []),
 CursoMateria(T1-O, ['L3', 'L4'], []),
 CursoMateria(T1-P, ['L1', 'L2'], []),
 CursoMater

In [58]:
profesores = profes.apply(lambda row: Profe(row['PROF #'], row['Bloques asignados'], row['Disponibilidad'], row['Capacidad']), axis=1).tolist()
profesores

[Profe(ID=PROF 1),
 Profe(ID=PROF 2),
 Profe(ID=PROF 3),
 Profe(ID=PROF 4),
 Profe(ID=PROF 5),
 Profe(ID=PROF 6),
 Profe(ID=PROF 7),
 Profe(ID=PROF 8),
 Profe(ID=PROF 9),
 Profe(ID=PROF 10),
 Profe(ID=PROF 11),
 Profe(ID=PROF 12),
 Profe(ID=PROF 13)]

# Definición de posibilidades

Acá informo a cada materia que profe puede dar la materia

Para cada materia, que profes podrían darla en base a disponibilidad y capacidad. Ordeno segun prioridad de materia (materias con menos profes posibles).

In [51]:
for materia in materias:
    for profe in profesores:
        if profe.podríaDarMateria(materia):
            materia.profesPosibles.append(profe)
materias

[CursoMateria(T7-A, ['M2', 'M3'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-B, ['J3', 'J4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-C, ['M3', 'X4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-D, ['V3', 'V4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-E, ['V1', 'V2'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T72-A, ['J1'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-B, ['L3'], [Profe(ID=PROF 3), Profe(ID=PROF 5)]),
 CursoMateria(T72-C, ['V3'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-D, ['J2'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-E, ['J4'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T1-A, ['L5', 'V3'], [Profe(ID=PROF 3), Profe(ID=PROF 4)]),
 CursoMateria(T1-B, ['M4', 'J1'], [Profe(ID=PROF 2), Profe(ID=PROF 3), Profe(ID=PROF 4), Profe(ID=PROF 9), Profe(ID=PROF 10)]),
 CursoMateria

In [52]:
materias.sort(key=lambda x: x.prioridad())
materias

[CursoMateria(T7-A, ['M2', 'M3'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-B, ['J3', 'J4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-C, ['M3', 'X4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-D, ['V3', 'V4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-E, ['V1', 'V2'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T72-B, ['L3'], [Profe(ID=PROF 3), Profe(ID=PROF 5)]),
 CursoMateria(T1-A, ['L5', 'V3'], [Profe(ID=PROF 3), Profe(ID=PROF 4)]),
 CursoMateria(T1-I, ['X1', 'V4'], [Profe(ID=PROF 3), Profe(ID=PROF 4)]),
 CursoMateria(T72-A, ['J1'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-C, ['V3'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-D, ['J2'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-E, ['J4'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T1-D, ['X2', 'J5'], [Profe(ID=PROF 3), Profe(ID=PROF 4

# Hago copias y voy asignando

In [64]:
materias_copy = copy.deepcopy(materias)
profesores_copy = copy.deepcopy(profesores)

Así no son los mismos, no me sirve. Debo copiar y asignar posibilidades cada vez...

In [65]:
for materia in materias_copy:
    for profe in profesores_copy:
        if profe.podríaDarMateria(materia):
            materia.profesPosibles.append(profe)
materias_copy.sort(key=lambda x: x.prioridad())
materias_copy

[CursoMateria(T7-A, ['M2', 'M3'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-B, ['J3', 'J4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-C, ['M3', 'X4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-D, ['V3', 'V4'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T7-E, ['V1', 'V2'], [Profe(ID=PROF 8), Profe(ID=PROF 12)]),
 CursoMateria(T72-B, ['L3'], [Profe(ID=PROF 3), Profe(ID=PROF 5)]),
 CursoMateria(T1-A, ['L5', 'V3'], [Profe(ID=PROF 3), Profe(ID=PROF 4)]),
 CursoMateria(T1-I, ['X1', 'V4'], [Profe(ID=PROF 3), Profe(ID=PROF 4)]),
 CursoMateria(T72-A, ['J1'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-C, ['V3'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-D, ['J2'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T72-E, ['J4'], [Profe(ID=PROF 3), Profe(ID=PROF 5), Profe(ID=PROF 12)]),
 CursoMateria(T1-D, ['X2', 'J5'], [Profe(ID=PROF 3), Profe(ID=PROF 4